<a href="https://colab.research.google.com/github/osebasp/ML-Applications-for-Actuarial-Science/blob/main/Estimaci%C3%B3n_reservas_de_siniestros_V01_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oscar Pulido
# Implementación modelo de Machine Learning para la estimación de la reserva de siniestros de un conjunto de datos de compañías aseguradoras V2.0 Parte 2

El objetivo de este cuaderno es desarrollar las fases de modelación del problema de estimación de reserva IBNR para el conjunto de datos analizado en el notebook anterior. Concretamente, en esta segunda parte del problema se retoma desde la fase de modelación. . Se usará este entorno de Pyhton, ya que es más eficiente para este tipo de labores que involucran Machine Learning.
Como ya se conoce el comportamiento de la base de datos no se hará la fase exploratoria en este notebook


#Método Chain Ladder


El primer paso es importar las librerías necesarias para implementar la solución.

In [1]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
!pip install chainladder
import chainladder as cl
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00


El siguiente paso es importar la información.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
MedMal = pd.read_csv("/content/drive/MyDrive/medmal_pos.csv")
MedMal

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_F2,CumPaidLoss_F2,BulkLoss_F2,EarnedPremDIR_F2,EarnedPremCeded_F2,EarnedPremNet_F2,Single,PostedReserve97_F2
0,669,Scpie Indemnity Co,1988,1988,1,121905,2716,97966,129104,-6214,135318,0,344558
1,669,Scpie Indemnity Co,1988,1989,2,112211,24576,64117,129104,-6214,135318,0,344558
2,669,Scpie Indemnity Co,1988,1990,3,103226,43990,39008,129104,-6214,135318,0,344558
3,669,Scpie Indemnity Co,1988,1991,4,99599,59722,20736,129104,-6214,135318,0,344558
4,669,Scpie Indemnity Co,1988,1992,5,96006,71019,13599,129104,-6214,135318,0,344558
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,44504,California Healthcare Ins Co Inc,1997,2002,6,3970,3971,-1,9711,5704,4007,1,15719
3396,44504,California Healthcare Ins Co Inc,1997,2003,7,3965,3965,0,9711,5704,4007,1,15719
3397,44504,California Healthcare Ins Co Inc,1997,2004,8,3965,3965,0,9711,5704,4007,1,15719
3398,44504,California Healthcare Ins Co Inc,1997,2005,9,3965,3965,0,9711,5704,4007,1,15719


Se hace una separación en datos de entrenamiento

In [4]:
MedMal_train = MedMal.head(3300).copy()
MedMal_train['GRNAME'].value_counts()

Scpie Indemnity Co                    100
National American Ins Co              100
Louisiana Med Mut Ins Co              100
Physicians Recip Insurers             100
Dentists Ins Co                       100
Seguros Triples Inc                   100
Utah Medical Ins Assoc                100
Medical Mut Ins Co Of ME              100
Preferred Professional Ins Co         100
National Guardian RRG Inc             100
Health Care Ind Inc                   100
MHA Ins Co                            100
State Volunteer Mut Ins Co            100
Texas Hospital Ins Exch               100
Nichido Fire & Marine Ins Co Ltd      100
Michigan Professional Ins Exch        100
NCMIC Ins Co                          100
Promutual Grp                         100
Underwriters At Lloyds London         100
Community Blood Cntr Exch RRG         100
Campmed Cas & Ind Co Inc MD           100
Homestead Ins Co                      100
Franklin Cas Ins Co RRG               100
MCIC VT Inc RRG                   

In [5]:
MedMal.columns

Index(['GRCODE', 'GRNAME', 'AccidentYear', 'DevelopmentYear', 'DevelopmentLag',
       'IncurLoss_F2', 'CumPaidLoss_F2', 'BulkLoss_F2', 'EarnedPremDIR_F2',
       'EarnedPremCeded_F2', 'EarnedPremNet_F2', 'Single',
       'PostedReserve97_F2'],
      dtype='object')

Se seleccionan las columnas de interés

In [6]:
MedMal_col_seleccionado = MedMal[MedMal.columns[0:6]]

Filtro para asegurar calidad de los datos

In [7]:
Lista_entidades_ceros = MedMal_col_seleccionado[MedMal_col_seleccionado[MedMal.columns[5]] <= 0]["GRCODE"].unique()

In [8]:
MedMal_filtrado = MedMal_col_seleccionado[~MedMal_col_seleccionado["GRCODE"].isin(Lista_entidades_ceros)]
MedMal_filtrado[MedMal_filtrado[MedMal.columns[5]] <= 0]["GRCODE"].unique() #Verificación debe estar en vacio

array([], dtype=int64)

Un ejercicio complementario involucra la estimación de las reservas de las compañías usando el método Chain Ladder. Previo a esto es necesario dar un vistazo a la información real, es decir, al triángulo con la información completa. En la siguiente parte de código se crea una clase que permite obtener distintos escenarios a partir de la información a partir de la información dada. Un triángulo obtenido a partir de los años de ocurrencia sin acumular, un caso acumulando los pago, un caso obteniendo la parte inferior del tríangulo acumulado, otro caso obteniendo la parte inferior, finalmente se tienen los pagos estimados asumiendo que no se conoce la información real. Adicional a esto, podemos ver la estimación de los factores de desarrollo y la reserva IBNR.

In [9]:
class ChainLadder: #clase se cálcula el Chain-Ladder
    def __init__(self, tabla = pd.DataFrame(), origin = "", development = "", columns = "", index = ""):

        self.tabla = tabla #OK
        self.origin = origin #OK
        self.development = development #OK
        self.index = index #OK
        self.columns = columns #OK

    def Triangulos(self):

        # Renombrar las columnas
        datos = self.tabla.rename(columns={self.origin: "AccidentYear", self.development: "DevelopmentLag", self.columns: "IncurLoss_C",
                                          self.index: "GRCODE"})

        diccionario_todos_triangulos = {}

        for k in datos["GRCODE"].unique():


            Filtro_datos = datos[datos["GRCODE"] == k] #se filtran los datos filtra por aseguradora

            #se crean los triangulos
            Triangulo_full = Filtro_datos.pivot_table(values = "IncurLoss_C", index = "AccidentYear", columns='DevelopmentLag', aggfunc="sum", margins=False)
            #se crea una copia
            Triangulo_full_acumulado = Triangulo_full.copy()
            #se guarda el numero de filas y columnas del triangulo
            num_filas = Triangulo_full_acumulado.shape[0]
            num_columnas = Triangulo_full_acumulado.shape[1]
            #se eliminan los datos del triangulo inferior
            Triangulo_full_mitad = Triangulo_full.copy()
            for i in range(num_filas):
                for j in range(1,i+1):
                    Triangulo_full_mitad.iloc[i, -j] = None  # Puedes establecerlo en None u otro valor si lo prefieres
            #se suman las columnas para hallar el tirangulo acumulado
            for indice, i in enumerate(range(1,num_columnas+1)):
                Triangulo_full_acumulado[Triangulo_full.columns[indice]] = Triangulo_full[Triangulo_full.columns[0:i]].sum(axis = 1)
            #se halla eliminan datos del triangulo inferior del triangulo acumulado
            Triangulo_acumulado_mitad = Triangulo_full_acumulado.copy()
            for i in range(num_filas):
                for j in range(1,i+1):
                    Triangulo_acumulado_mitad.iloc[i, -j] = None  # Puedes establecerlo en None u otro valor si lo prefieres
            #se calculan los factores
            factores0 = Triangulo_acumulado_mitad.sum(axis = 0) # Rojo
            factores1 = Triangulo_acumulado_mitad.sum(axis = 0)-np.flip(np.diag(np.fliplr(Triangulo_acumulado_mitad), 0)) # Azul
            factores0 = factores0[1:10]
            factores1 = factores1[0:-1]
            factores = factores0.reset_index(drop = True) / factores1.reset_index(drop = True)
            #se estima el triangulo por chain-ladder
            Triangulo_estimado = Triangulo_acumulado_mitad.copy()
            for i in list(reversed(range(num_filas))):
                comodin = np.diag(np.fliplr(Triangulo_acumulado_mitad), 0)[i]
                for j in range(1,i+1):
                    Triangulo_estimado.iloc[i, -j] = comodin*factores.iloc[-i+9:-j+10].prod()   # Puedes establecerlo en None u otro valor si lo prefieres


            #se calcula la reserva
            reserva_total = sum(np.array(list(reversed(np.array(Triangulo_estimado[10]))))-np.flip(np.diag(np.fliplr(Triangulo_estimado), 0)))
            #se crea diccionario que va ser el resultado final
            diciconario_triangulo = {'Triangulo_full':Triangulo_full, "Triangulo_full_mitad":Triangulo_full_mitad, "Triangulo_full_acumulado":Triangulo_full_acumulado,
                                     "Triangulo_acumulado_mitad":Triangulo_acumulado_mitad, "factores":factores, "Triangulo_estimado":Triangulo_estimado,
                                     "reserva_total":reserva_total}

            nombre = k
            diccionario_todos_triangulos[nombre] = diciconario_triangulo

        return diccionario_todos_triangulos

A continuación, una clase para un segundo escenario

In [10]:
class ChainLadder_corto: #clase se cálcula el Chain-Ladder corto
    def __init__(self, tabla = pd.DataFrame(), origin = "", development = "", columns = "", index = ""):

        self.tabla = tabla #OK
        self.origin = origin #OK
        self.development = development #OK
        self.index = index #OK
        self.columns = columns #OK

    def Triangulos(self):

        # Renombrar las columnas
        datos = self.tabla.rename(columns={self.origin: "AccidentYear", self.development: "DevelopmentLag", self.columns: "IncurLoss_C",
                                          self.index: "GRCODE"})

        diccionario_todos_triangulos = {}

        for k in datos["GRCODE"].unique():

            Filtro_datos = datos[datos["GRCODE"] == k]#se filtran los datos filtra por aseguradora
            #se crean los triangulos
            Triangulo_full = Filtro_datos.pivot_table(values = "IncurLoss_C", index = "AccidentYear", columns='DevelopmentLag', aggfunc="sum", margins=False)
            #se crea una copia
            Triangulo_full_acumulado = Triangulo_full.copy()
            #se guarda el numero de filas y columnas del triangulo
            num_filas = Triangulo_full_acumulado.shape[0]
            num_columnas = Triangulo_full_acumulado.shape[1]
            #se eliminan los datos del triangulo inferior
            Triangulo_full_mitad = Triangulo_full.copy()
            for i in range(num_filas):
                for j in range(1,i+1):
                    Triangulo_full_mitad.iloc[i, -j] = None  # Puedes establecerlo en None u otro valor si lo prefieres
            #se crea diccionario que va ser el resultado final
            diciconario_triangulo = {'Triangulo_full':Triangulo_full, "Triangulo_full_mitad":Triangulo_full_mitad}

            nombre = k
            diccionario_todos_triangulos[nombre] = diciconario_triangulo

        return diccionario_todos_triangulos


Se hace una configuración adicional

In [11]:
MedMal_filtrado["GRCODE"].unique()

array([  669,   683,  7854, 32514, 33049, 33111, 36234, 36277, 36676,
       40568, 40975, 41467, 43656, 43770])

Se almacena en una variable

In [12]:
codigo_aseguradora = MedMal_filtrado["GRCODE"].unique()[0] #se filtra por la primera aseguradora para el ejemplo
print("Se realizará un ejemplo con la aseguradora:", codigo_aseguradora)

Se realizará un ejemplo con la aseguradora: 669


In [13]:
resultados = ChainLadder(tabla = MedMal_filtrado, origin = "AccidentYear", development = "DevelopmentLag", columns = "IncurLoss_F2", index = "GRCODE")
triangulos_resultados = resultados.Triangulos()

A continuación, se prueba el ejercicio para la compañía 669

In [14]:
triangulos_resultados[codigo_aseguradora]["Triangulo_full_acumulado"]

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,121905,234116,337342,436941,532947,623434,706074,786480,865400,943911
1989,122679,235844,345881,447023,537840,619759,697250,770827,843543,915860
1990,118157,235654,352031,451926,541178,623094,702228,778561,854173,929523
1991,117981,240424,361480,475275,578105,676176,771046,862108,952601,1042946
1992,131059,261214,385409,499383,606200,705382,797970,888970,978226,1067477
1993,134700,265457,390710,505427,616721,714735,811607,907321,1003338,1099385
1994,136749,264941,386296,498173,594325,685827,776325,868195,960043,1051981
1995,140962,273367,391699,491749,580558,662918,744904,826791,908587,990369
1996,134473,263453,377098,481371,580647,678429,775711,873449,971050,1068301


En esta parte podemos "partir" el triángulo en parte inferior y superior. Obtener la parte inferior implica que podrémos construir una métrica que permita medir el desemepeño de las implementaciones por Chain Ladder determínistico y el método de regresión lineal. En la parte que sigue se obtendrá la parte superior (conocida)

In [15]:
triangulos_resultados[codigo_aseguradora]["Triangulo_acumulado_mitad"]

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,121905,234116.0,337342.0,436941.0,532947.0,623434.0,706074.0,786480.0,865400.0,943911.0
1989,122679,235844.0,345881.0,447023.0,537840.0,619759.0,697250.0,770827.0,843543.0,NaN
1990,118157,235654.0,352031.0,451926.0,541178.0,623094.0,702228.0,778561.0,NaN,NaN
1991,117981,240424.0,361480.0,475275.0,578105.0,676176.0,771046.0,NaN,NaN,NaN
1992,131059,261214.0,385409.0,499383.0,606200.0,705382.0,NaN,NaN,NaN,NaN
1993,134700,265457.0,390710.0,505427.0,616721.0,NaN,NaN,NaN,NaN,NaN
1994,136749,264941.0,386296.0,498173.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,140962,273367.0,391699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,134473,263453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En la parte que sigue se obtendrá la parte inferior (a predecir)

In [16]:
triangulos_resultados[codigo_aseguradora]["Triangulo_estimado"]

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,121905,234116.00000,337342.000000,436941.000000,532947.000000,623434.000000,706074.000000,786480.000000,865400.000000,9.439110e+05
1989,122679,235844.00000,345881.000000,447023.000000,537840.000000,619759.000000,697250.000000,770827.000000,843543.000000,9.200711e+05
1990,118157,235654.00000,352031.000000,451926.000000,541178.000000,623094.000000,702228.000000,778561.000000,854369.993215,9.318803e+05
1991,117981,240424.00000,361480.000000,475275.000000,578105.000000,676176.000000,771046.000000,855386.937928,938676.522910,1.023835e+06
1992,131059,261214.00000,385409.000000,499383.000000,606200.000000,705382.000000,798084.554401,885383.107099,971593.438671,1.059739e+06
1993,134700,265457.00000,390710.000000,505427.000000,616721.000000,716316.455938,810456.035945,899107.844295,986654.562494,1.076166e+06
1994,136749,264941.00000,386296.000000,498173.000000,603790.859451,701298.202217,793464.056665,880257.195792,965968.414031,1.053603e+06
1995,140962,273367.00000,391699.000000,507257.864803,614801.810119,714087.332406,807933.957043,896309.887559,983584.166882,1.072817e+06
1996,134473,263453.00000,386575.428325,500622.739291,606759.968961,704746.798910,797365.874057,884585.813839,970718.512445,1.058784e+06


In [17]:
Reserva = triangulos_resultados[codigo_aseguradora]["reserva_total"]
Reserva

4278625.832135844

#Método por regresión lineal

El mayor proósito de estos cuadernos se abarca en la presente sección. La idea es intentar hacer estimaciones de pagos futuros por concepto de siniestros usando el método de regresión lineal. Particularmente, se puede pensar en un modelo multiplicativo donde tenemos coeicientes Cij con i=accidentes y j= desarrollo. De ese modo, la idea es plantear un modelo que multiplique dos cosas, una va a depender de las filas, es decir de los años de accidentalidad, y la otra dependerá de las columnas, es decir de los años de desarrollo. En ese sentido, es necesario estimar esos coeficientes multiplicativos que denominaremos U y S por el momento. Para eso, se puede pensar en un modelo de la forma ln(E(Zij))=ln(Ui)+ln(Sj), notar que este modelo de elasticidades se planteó de esa manera para poder convertir los productos del modelo multiplictivo en la suma de parámetros.

Primero se explorará un enfoque con planteamiento del modelo multiplicativo. En segundo lugar se abarcará un método que involucré la construcción de una librería que generé una regresión Lasso, otra para regresión Ridge y otra para regresión lineal. La evaluación de los resultados se presenta en la siguiente sección de este cuaderno.

#Bases modelo multiplicativo
Importación de librerías

In [18]:
import itertools
import re
import math

Luego se una función que permita plantear el modelo multiplicativo, primero se hace una función para las columnas (J)

In [19]:
def columnas(valores,variable):
    y = [re.findall("\\d+", j)[0] for j in valores]
    y = [int(i) for i in y]
    todas = list(set(y))
    df = pd.DataFrame()
    df[f"y_{variable}"] = y
    for k in todas:
        #print(k)
        df[f"{variable}_{k}"] = ([1 if k == j else 0 for j in y])
    return df


A continuación, se preparará una función para la construucción de unas matrices que permita usar el modelo multiplicativo

In [20]:
def matrix_X(df_triangulo):
    k = len(df_triangulo.columns)
    alpha = [f'a_{i}' for i in range(1,k+1)]
    mu    = [f'u_{i}' for i in range(1,k+1)]
    lists = [alpha, mu]
    df    = pd.DataFrame(list(itertools.product(*lists)), columns=['a', 'u'])

    alpha    = columnas(valores  = df.a, variable = 'a')
    mu       = columnas(valores=df.u, variable = 'u')
    df_col= pd.concat([alpha, mu], axis=1)


    df_col['y_a'] = df_col['y_a'].astype(str) + df_col['y_u'].astype(str)
    df_col['y_a'] = [int(i) for i in df_col['y_a']]
    df_col = df_col.drop(['y_u', 'u_1'], axis=1)
    df_col['a_1'] = 1
    df_col.rename(columns={'a_1': 'b0'}, inplace=True)
    df_col.rename(columns={'y_a': 'y_ii'}, inplace=True)
    #df_col = df_col.drop(['y_ii'], axis=1)
    return df_col


def matrix_y(df_triangulo):
    k = len(df_triangulo.columns)
    d0 = pd.DataFrame()
    for i in range(k):
        for j in range(k):
            d1 = pd.DataFrame({'y_ii': [int(f'{i+1}{j+1}')], 'Y': [math.log(df_triangulo.iloc[i, j])]})
            d0 = pd.concat([d0, d1], axis=0)
    return d0



Finalmente, una función que permita obtener el triángulo run-off y servir como insumo para el modelo multiplicativo

In [21]:
def triangulo(df, grcode, entreno):

    if entreno:
        df_trinagulo = df[(df['GRCODE']== grcode ) & (df['DevelopmentYear']<=1997)].copy()
    else:
        df_trinagulo = df[df['GRCODE']== grcode].copy()

    df_g         = df_trinagulo.groupby(["AccidentYear", "DevelopmentLag"]).agg({'IncurLoss_F2': ['max']})
    df_g.columns = ['Pagos']
    df_g         = df_g.reset_index()
    pivot_data   = df_g.pivot(index='AccidentYear',columns='DevelopmentLag',values='Pagos').reset_index()
    pivot_data   = pivot_data.drop('AccidentYear', axis=1).cumsum(axis=1)

    return pivot_data

A continuación, se prepara la data de entrenemiento y prueba para el modelo. Se seguirá trabajando sobre la compañía con código 7854.

In [22]:
data_training = triangulo(MedMal, grcode=7854, entreno=True)
data_test  = triangulo(MedMal, grcode=7854, entreno=False)
data_training

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
0,10760.0,22255.0,33550.0,41531.0,47969.0,53651.0,59312.0,64955.0,70546.0,76137.0
1,8699.0,17218.0,25667.0,30782.0,35235.0,39621.0,43747.0,47855.0,51963.0,NaN
2,8548.0,17344.0,26408.0,34390.0,42246.0,49863.0,57190.0,64517.0,NaN,NaN
3,9248.0,17424.0,25089.0,32685.0,39432.0,46001.0,52686.0,NaN,NaN,NaN
4,9415.0,18059.0,30154.0,42811.0,54875.0,66552.0,NaN,NaN,NaN,NaN
5,11480.0,28188.0,46439.0,65875.0,84071.0,NaN,NaN,NaN,NaN,NaN
6,19035.0,37962.0,57085.0,75015.0,NaN,NaN,NaN,NaN,NaN,NaN
7,23479.0,53573.0,83858.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,33824.0,69403.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,22860.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Posteriormente, se hace un procedmiento adicional para el conjunto de datos de entrenamiento, lo cual permitirá tener una visión preliminar del modelo multiplicativo.

In [23]:
Y = matrix_y(data_training)
X = matrix_X(data_training)

Y_X          = pd.merge(Y, X, on='y_ii', how='inner')
data_entrenamiento = Y_X[Y_X['Y'].notna()]
data_entrenamiento = data_entrenamiento.drop(['y_ii'], axis=1)
data_entrenamiento.head()

,Y,b0,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,u_10
0,9.283591,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10.010322,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,10.420792,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,10.634195,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,10.778310,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


Otra consideración adicional para los datos de prueba.

In [24]:
Y_prueba      = matrix_y(data_test)
x_prueba      = Y_X[Y_X['Y'].isna()].drop(['Y'], axis=1)
data_prueba_  = pd.merge(Y_prueba, x_prueba, on='y_ii', how='inner')
data_prueba=data_prueba_.drop(['y_ii'], axis=1)
y_ii = data_prueba_['y_ii']

Finalmente se muestra el modelo, se empieza importando algunas librerías esenciales

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

Poniendo el modelo en práctica

In [26]:
x_entrenamiento = data_entrenamiento.drop('Y', axis=1)
y_entrenamiento = data_entrenamiento['Y']
x_prueba  = data_prueba.drop('Y', axis=1)
y_prueba  = data_prueba['Y']


model = LinearRegression()
model.fit(x_entrenamiento, y_entrenamiento)

y_pred = model.predict(x_prueba)
y_pred

array([10.95541449, 11.00841678, 11.09511511, 10.92560088, 10.95848732,
       11.04518565, 11.03227818, 11.12094847, 11.15383492, 11.24053324,
       11.30848663, 11.38822684, 11.47689713, 11.50978357, 11.5964819 ,
       11.43752839, 11.55604754, 11.63578775, 11.72445804, 11.75734449,
       11.84404281, 11.55891715, 11.75561435, 11.87413351, 11.95387371,
       12.04254401, 12.07543045, 12.16212878, 11.58459892, 11.8494731 ,
       12.04617029, 12.16468945, 12.24442966, 12.33309995, 12.3659864 ,
       12.45268472, 10.76315544, 11.19644298, 11.46131715, 11.65801435,
       11.77653351, 11.85627372, 11.94494401, 11.97783045, 12.06452878])

#Método regresión Lasso y regresión Ridge

Antes, y de cara al ejercicio posterior de diseño de experimento por Leave One Out Cross Validation, es necesario crear una matriz con valores de 0 mediante el siguiente código:

In [27]:
datos = MedMal_filtrado
size = triangulos_resultados[datos["GRCODE"].unique()[0]]["Triangulo_acumulado_mitad"].shape[0]#10
size1 =  size**2 #100
size2 = (size1-size)/2 + size #55
cant_parametros = size*2-1 #19
matriz_de_ceros = np.zeros((size1, cant_parametros)) #matriz de ceros
matriz_de_ceros[:,0] = 1 #columna 0 se llene de unos
for i in range(size): #recorre las columnas
    for j in range(size): #recorre las filas
        k = i*10+j
        if i != 0:
            matriz_de_ceros[k,i] = 1
        if j != 0:
            matriz_de_ceros[k,j+9] = 1
matriz_de_ceros = pd.DataFrame(matriz_de_ceros)

Acá se implementa una clase  para realizar una regresión lineal con opciones de regularización L1 (Lasso) y L2 (Ridge). Como se verá, la clase tiene tres métodos principales. En el primero, que es regresión lineal se ejecutan tareas como realizar la regresión lineal estandar y con regularización Ridge y Lasso. Asimismo, se calculan métricas de error de Error Cuadrático Medio y Error de Porcentaje Absoluto. El segundo método realiza predicciones a partir de un nuevo conjunto de datos, sin embargo, en términos generales realiza tareas muy similares a las del primer método. Finalmente, el tercer método evalúa métricas del mejor modelo que escogieron los datos de validación para saber como se comportan en datos de test

In [29]:
class Reserva_Regresion_lineal:
    def __init__(self, tabla = pd.DataFrame(), origin = "", development = "", columns = "", index = "", alpha = 1, matriz_de_ceros = pd.DataFrame()):

        self.tabla = tabla #OK
        self.origin = origin #OK
        self.development = development #OK
        self.index = index #OK
        self.columns = columns #OK
        self.alpha = alpha #OK
        self.matriz_de_ceros = matriz_de_ceros #OK


    def Regresion_lineal(self):

        # Renombrar las columnas
        datos = self.tabla.rename(columns={self.origin: "AccidentYear", self.development: "DevelopmentLag", self.columns: "IncurLoss_C",
                                          self.index: "GRCODE"})
        inicio = time.time()
        #se aplica el algoritmo de chain-ladder corto para poder entrenar los modelos
        resultados = ChainLadder_corto(tabla = datos, origin = "AccidentYear", development = "DevelopmentLag", columns = "IncurLoss_C", index = "GRCODE")
        triangulos_resultados = resultados.Triangulos()
        fin = time.time()
        #print(fin-inicio)

        #triangulos_resultados

        inicio = time.time()
        Y = pd.DataFrame()
        Y_completo = pd.DataFrame()
        matriz_X = pd.DataFrame()
        #se pegan las matrices X's como tantas aseguradoras se necesiten para el entrenamiento
        for i in datos["GRCODE"].unique():#[3492]:#
            triangulo = triangulos_resultados[i]["Triangulo_full_mitad"]
            triangulo_completo = triangulos_resultados[i]["Triangulo_full"]
            triangulo_log =  np.log(triangulo) #se aplica logaritmo a los datos
            triangulo_completo_log = np.log(triangulo_completo) #se aplica logaritmo a los datos
            vector_Y = pd.melt(triangulo_log)
            vector_Y_completo = pd.melt(triangulo_completo_log)
            vector_Y["GRCODE"] = i
            Y = pd.concat([Y,vector_Y], axis = 0)
            Y_completo = pd.concat([Y_completo,vector_Y_completo], axis = 0)
            matriz_X = pd.concat([matriz_X, self.matriz_de_ceros], axis = 0)

        fin = time.time()
        #print(fin-inicio)

        inicio = time.time()

        matriz_X = matriz_X.reset_index(drop=True)
        Y = Y.reset_index(drop=True)
        Y_completo = Y_completo.reset_index(drop=True)
        Y_completo = Y_completo[["value"]]
        Y_completo.columns = ["value_completo"]

        matriz_regresion = pd.concat([Y[["value", "GRCODE"]], Y_completo, matriz_X], axis = 1)
        matriz_regresion = matriz_regresion.reset_index(drop=True)
        matriz_regresion1 = matriz_regresion[~matriz_regresion["value"].isnull()]
        #se cambian los nombres por lo nombres correctos de los parametros
        matriz_regresion1.columns = ['Z', "GRCODE", "Z_completo", "u", "alpha1", "alpha2", "alpha3", "alpha4", "alpha5",
                                    "alpha6", "alpha7", "alpha8", "alpha9", "beta1", "beta2", "beta3",
                                    "beta4", "beta5", "beta6", "beta7", "beta8", "beta9"]
        matriz_regresion1 = matriz_regresion1.reset_index(drop=True)
        X = matriz_regresion1.iloc[:,3:]
        Y = matriz_regresion1.iloc[:,0]



        #linear regresion sklearn

        ###### Se aplican o entrenan las diferentes regresiones
        self.Regresion_lineal1 = LinearRegression() #aplicación regresión lineal
        self.Regresion_lineal1.fit(np.array(X), np.array(Y)) #entrenamiento regresión lineal
        self.B = self.Regresion_lineal1.coef_ #coeficientes regresión lineal
        #ridge sklearnr
        self.ridge_model = Ridge(alpha = self.alpha) #aplicación regresión de ridge
        self.ridge_model.fit(np.array(X), np.array(Y)) #entrenamiento regresión de ridge
        self.B_ridge_1 = self.ridge_model.coef_ #coeficientes regresión de ridge
        ##################### lasso
        self.lasso_model = Lasso(alpha = self.alpha) #aplicación regresión de lasso
        self.lasso_model.fit(np.array(X), np.array(Y)) #entrenamiento regresión de lasso
        self.B_lasso = self.lasso_model.coef_ #coeficientes regresión de lasso


        #se hallan los coeficientes
        coeficientes = {"Coef_normal":self.B, "Coef_ridge1":self.B_ridge_1, "Coef_lasso":self.B_lasso}

        #diccionario_resultados = {"Metricas MSE":metricas_MSE, "Metricas MAPE":metricas_MAPE, "coeficientes":coeficientes ,"Comparacion":comparacion, "X":X}
        diccionario_resultados = {"coeficientes":coeficientes, "X":X}

        fin = time.time()
        #print(fin-inicio) #cuanto demora el codigo

        return diccionario_resultados

    def predict(self, datos_new):
        #Se aplican los resultados de entrenamiendo a conjunto de validación

        #inicio = time.time()

        datos_new = datos_new.rename(columns={self.origin: "AccidentYear", self.development: "DevelopmentLag", self.columns: "IncurLoss_C",
                                          self.index: "GRCODE"})
        #se aplica el algoritmo de chain-ladder corto para poder entrenar los modelos
        resultados = ChainLadder_corto(tabla = datos_new, origin = "AccidentYear", development = "DevelopmentLag", columns = "IncurLoss_C", index = "GRCODE")

        triangulos_resultados = resultados.Triangulos()

        i = datos_new["GRCODE"].unique()[0]#[3492]:# Se filtran los datos para la aseguradora de validación
        triangulo = triangulos_resultados[i]["Triangulo_full_mitad"] #se calcula triangulo de arriba
        triangulo_completo = triangulos_resultados[i]["Triangulo_full"] #se calcula triangulo completo o matriz completa
        triangulo_log =  np.log(triangulo) #se saca logaritmo a los datos
        triangulo_completo_log = np.log(triangulo_completo) #se saca logaritmo a los datos
        vector_Y = pd.melt(triangulo_log) #se pasa el triangulo a vector
        vector_Y_completo = pd.melt(triangulo_completo_log) #se pasa el triangulo a vector
        vector_Y["GRCODE"] = i
        Y = vector_Y
        Y_completo = vector_Y_completo

        Y = Y.reset_index(drop=True)
        Y_completo = Y_completo.reset_index(drop=True)
        Y_completo = Y_completo[["value"]]
        Y_completo.columns = ["value_completo"]

        X = self.matriz_de_ceros

        matriz_new = pd.concat([Y[["value", "GRCODE"]], Y_completo, X], axis = 1)
        matriz_new = matriz_new.reset_index(drop=True)

        Y_test = matriz_new[matriz_new["value"].isnull()].iloc[:,2] #matriz X de datos
        X_test = matriz_new[matriz_new["value"].isnull()].iloc[:,3:] #vector Y de datos

        #Y_Ajustado = np.dot(X_test,self.B)
        Y_Ajustado = self.Regresion_lineal1.predict(X_test) #Se predice regresion lineal normal para datos de validación
        #Y_Ajustado_ridge = np.dot(X_test,self.B_ridge)
        Y_Ajustado_ridge_1 = self.ridge_model.predict(X_test) #Se predice regresion lineal de ridge para datos de validación
        Y_Ajustado_lasso = self.lasso_model.predict(X_test) #Se predice regresion lineal de lasso para datos de validación

        comparacion = pd.DataFrame([np.array(Y_test), Y_Ajustado, Y_Ajustado_ridge_1, Y_Ajustado_lasso]).T #Comparacion de Y y Y-ajustado
        comparacion.columns = ["Y_test", "Y_ajustado", "Y_ajustado_ridge_1", "Y_ajustado_lasso"]
        #metrica
        comparacion["GRCODE"] = np.array(i)

        #calculo metrica MSE
        MSE = ((comparacion["Y_test"] - comparacion["Y_ajustado"])**2).mean()
        #MSE_ridge = ((comparacion["Y_test"] - comparacion["Y_ajustado_ridge"])**2).mean()
        MSE_ridge_1 = ((comparacion["Y_test"] - comparacion["Y_ajustado_ridge_1"])**2).mean()
        MSE_lasso = ((comparacion["Y_test"] - comparacion["Y_ajustado_lasso"])**2).mean()

        #calculo metrica MAPE
        MAPE = abs((comparacion["Y_test"] - comparacion["Y_ajustado"])/comparacion["Y_test"]).mean()*100
        #MAPE_ridge = abs((comparacion["Y_test"] - comparacion["Y_ajustado_ridge"])/comparacion["Y_test"]).mean()*100
        MAPE_ridge_1 = abs((comparacion["Y_test"] - comparacion["Y_ajustado_ridge_1"])/comparacion["Y_test"]).mean()*100
        MAPE_lasso = abs((comparacion["Y_test"] - comparacion["Y_ajustado_lasso"])/comparacion["Y_test"]).mean()*100

        metricas_MSE = {"MSE":MSE, "MSE_ridge_1":MSE_ridge_1, "MSE_lasso":MSE_lasso}
        metricas_MAPE = {"MAPE":MAPE, "MAPE_ridge_1":MAPE_ridge_1, "MAPE_lasso":MAPE_lasso}

        coeficientes = {"Coef_normal":self.B, "Coef_ridge1":self.B_ridge_1, "Coef_lasso":self.B_lasso}

        diccionario_resultados_test = {"Metricas MSE":metricas_MSE, "Metricas MAPE":metricas_MAPE, "coeficientes":coeficientes ,"Comparacion":comparacion, "X":X}

        #Acá se escoge el Mejor modelo de los 3 que hay
        metricas_model1 = pd.DataFrame(metricas_MAPE, index = [0]).T
        metricas_model1.index = ['MAPE', 'MAPE_ridge_1', 'MAPE_lasso']
        metricas_model1.columns = ["MAPE"]
        mejor_modelo = metricas_model1[metricas_model1["MAPE"] == metricas_model1["MAPE"].sort_values()[0]]
        #mejor_modelo = str(mejor_modelo.iloc[0,0])
        #print(mejor_modelo.index[0])

        modelo = {'MAPE':self.Regresion_lineal1, 'MAPE_ridge_1':self.ridge_model , 'MAPE_lasso':self.lasso_model}
        coeficientes_orden = {'MAPE':"Coef_normal", 'MAPE_ridge_1':"Coef_ridge1" , 'MAPE_lasso':"Coef_lasso"}
        nombres_orden = {'MAPE':"Regresión normal", 'MAPE_ridge_1':"Regresión de Ridge" , 'MAPE_lasso':"Regresión de Lasso"}
        nombre_mejor_modelo = mejor_modelo.index[0] #mejor modelo

        #resultados mejor modelo
        self.mejor_modelo_resultado = {"nombre mejor modelo":  nombres_orden[nombre_mejor_modelo],
                                       "MAPE mejor modelo":metricas_MAPE[nombre_mejor_modelo],
                                       "coeficientes mejor modelo": coeficientes[coeficientes_orden[nombre_mejor_modelo]],
                                       "mejor modelo":modelo[nombre_mejor_modelo]}

        #fin = time.time()
        #print(fin-inicio)

        return diccionario_resultados_test, self.mejor_modelo_resultado

    def predict_test(self, datos_test):

        #inicio = time.time()
        #datos de test
        datos_new = datos_test.rename(columns={self.origin: "AccidentYear", self.development: "DevelopmentLag", self.columns: "IncurLoss_C",
                                          self.index: "GRCODE"})

        #se aplcia la clase chainlader corto
        resultados = ChainLadder_corto(tabla = datos_new, origin = "AccidentYear", development = "DevelopmentLag", columns = "IncurLoss_C", index = "GRCODE")

        triangulos_resultados = resultados.Triangulos()

        i = datos_new["GRCODE"].unique()[0]#[3492]:## Se filtran los datos para la aseguradora de TEST
        triangulo = triangulos_resultados[i]["Triangulo_full_mitad"] #se calcula triangulo de arriba
        triangulo_completo = triangulos_resultados[i]["Triangulo_full"] #se calcula triangulo completo o matriz completa
        triangulo_log =  np.log(triangulo)#se saca logaritmo a los datos
        triangulo_completo_log = np.log(triangulo_completo)#se saca logaritmo a los datos
        vector_Y = pd.melt(triangulo_log)#se pasa el triangulo a vector
        vector_Y_completo = pd.melt(triangulo_completo_log)#se pasa el triangulo a vector
        vector_Y["GRCODE"] = i
        Y = vector_Y
        Y_completo = vector_Y_completo

        Y = Y.reset_index(drop=True)
        Y_completo = Y_completo.reset_index(drop=True)
        Y_completo = Y_completo[["value"]]
        Y_completo.columns = ["value_completo"]

        X = self.matriz_de_ceros

        matriz_new = pd.concat([Y[["value", "GRCODE"]], Y_completo, X], axis = 1)
        matriz_new = matriz_new.reset_index(drop=True)

        Y_test = matriz_new[matriz_new["value"].isnull()].iloc[:,2] #vector Y de datos
        X_test = matriz_new[matriz_new["value"].isnull()].iloc[:,3:] #matriz X de datos

        Y_Ajustado = self.mejor_modelo_resultado["mejor modelo"].predict(X_test)#se ajusta datos de test según el mejor modelo que definió la validación


        comparacion = pd.DataFrame([np.array(Y_test), Y_Ajustado]).T #comparacion entre Y y Y-ajustado
        comparacion.columns = ["Y_test", "Y_ajustado modelo final"]
        #metrica
        comparacion["GRCODE"] = np.array(i)

        #se calcula mse para conjunto de test
        MSE = ((comparacion["Y_test"] - comparacion["Y_ajustado modelo final"])**2).mean()


        #se calcula mape para conjunto de test
        MAPE = abs((comparacion["Y_test"] - comparacion["Y_ajustado modelo final"])/comparacion["Y_test"]).mean()*100


        metricas_MSE = {"MSE modelo final":MSE}
        metricas_MAPE = {"MAPE modelo final":MAPE}

        #coeficientes = {"Coef_normal":self.mejor_modelo_resultado.coef_}

        #se hallan las metricas del mejor modelo para el conjunto de test
        resultados_modelo_final = {"nombre mejor modelo": self.mejor_modelo_resultado["nombre mejor modelo"],
                                   "Metricas MSE": metricas_MSE, "Metricas MAPE":metricas_MAPE,
                                   "coeficientes": self.mejor_modelo_resultado["coeficientes mejor modelo"],
                                   "Comparacion": comparacion, "X":X,
                                   "mejor modelo":self.mejor_modelo_resultado["mejor modelo"]}

        return resultados_modelo_final

A continuación, se filtra la base de datos para visualizar los resultados de las compañías.

In [30]:
MedMal_filtrado_unico = MedMal_filtrado[MedMal_filtrado["GRCODE"] == MedMal_filtrado["GRCODE"].unique()[0]] #conjunto de validación (una aseguradora)
MedMal_filtrado_unico1 = MedMal_filtrado[MedMal_filtrado["GRCODE"] == MedMal_filtrado["GRCODE"].unique()[1]] #conjunto de testeo (una aseguradora)
MedMal_filtrado #Dataframe completo con todas las aseguradoras

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_F2
0,669,Scpie Indemnity Co,1988,1988,1,121905
1,669,Scpie Indemnity Co,1988,1989,2,112211
2,669,Scpie Indemnity Co,1988,1990,3,103226
3,669,Scpie Indemnity Co,1988,1991,4,99599
4,669,Scpie Indemnity Co,1988,1992,5,96006
...,...,...,...,...,...,...
3295,43770,Clinic Mut Ins Co RRG,1997,2002,6,886
3296,43770,Clinic Mut Ins Co RRG,1997,2003,7,732
3297,43770,Clinic Mut Ins Co RRG,1997,2004,8,732
3298,43770,Clinic Mut Ins Co RRG,1997,2005,9,709


Con base en la clase anterior se ejecuta la regresión lineal

In [31]:
resultados1 = Reserva_Regresion_lineal(tabla = MedMal_filtrado, origin = "AccidentYear", development = "DevelopmentLag", columns = MedMal_filtrado.columns[5],
                                        index = "GRCODE", alpha = 0.001, matriz_de_ceros = matriz_de_ceros)
resultados_regresion = resultados1.Regresion_lineal()

Los resultados de la evaluación de los modelos presentados en el modelo anterior se muestran en la siguiente sección.

#5. Fase de evaluación

En esta parte del proyecto, se evaluarán los modelos siguiendo las métricas de Error Cuadrático Medio, Error Absoluto Porcentual Medio y se verá una implementación de un diseño de prueba usando los resultados de las validaciones por medio de Leave One Out Cross Validation. Primero se importan algunas librerías necesarias.

In [32]:
from sklearn.metrics import mean_squared_error
import numpy as np

#Modelo determinístico

Primero centrarémos la evaluación del modelo por Chain Ladder (determinísico). Vale la pena recordar que tenemos un conjunto de información real que nos permite verificar la precisión de la información.  Se quiere determinar la diferencia porcentual entre las predicciones y la información real para el caso particular de la compañía Markel Corp Grp. La manera de proceder es con una tasa de error porcentual que  permitirá evaluar las diferencias por cada uno de los elementos estimados de cada elemento de año de accidente vs año de desarrollo

In [33]:
Info_Real_1=triangulos_resultados[7854]["Triangulo_full_acumulado"]
Info_Real_1

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,10760,22255,33550,41531,47969,53651,59312,64955,70546,76137
1989,8699,17218,25667,30782,35235,39621,43747,47855,51963,56071
1990,8548,17344,26408,34390,42246,49863,57190,64517,71844,79171
1991,9248,17424,25089,32685,39432,46001,52686,59386,66137,72565
1992,9415,18059,30154,42811,54875,66552,77662,88945,100263,111411
1993,11480,28188,46439,65875,84071,102027,119757,137239,154522,171865
1994,19035,37962,57085,75015,91865,108101,124533,139983,155477,170771
1995,23479,53573,83858,113479,142801,171122,198549,225976,253361,280757
1996,33824,69403,104814,136466,166080,193008,220290,247487,274614,301434


A continuación se muestran las estimaciones obtenidas con el método Chain Ladder. Recordar que se creó una clase para realizar este procedmiento.

In [34]:
Info_Estimada_1=triangulos_resultados[7854]["Triangulo_estimado"]
Info_Estimada_1

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,10760,22255.000000,33550.000000,41531.000000,47969.000000,53651.000000,59312.000000,64955.000000,70546.000000,76137.000000
1989,8699,17218.000000,25667.000000,30782.000000,35235.000000,39621.000000,43747.000000,47855.000000,51963.000000,56081.236796
1990,8548,17344.000000,26408.000000,34390.000000,42246.000000,49863.000000,57190.000000,64517.000000,70063.940723,75616.735957
1991,9248,17424.000000,25089.000000,32685.000000,39432.000000,46001.000000,52686.000000,58300.833840,63313.330847,68331.118288
1992,9415,18059.000000,30154.000000,42811.000000,54875.000000,66552.000000,74926.244184,82911.257496,90039.670637,97175.607452
1993,11480,28188.000000,46439.000000,65875.000000,84071.000000,97816.887963,110125.195829,121861.419421,132338.628063,142826.894861
1994,19035,37962.000000,57085.000000,75015.000000,91874.430291,106896.205046,120346.964203,133172.538494,144622.236667,156084.019407
1995,23479,53573.000000,83858.000000,110861.228526,135777.007428,157976.999482,177855.259627,196809.587728,213730.571607,230669.414714
1996,33824,69403.000000,107448.412436,142048.021726,173972.953010,202418.109135,227888.398130,252174.840250,273855.930362,295559.903750


A continuación se calculan las diferencias porcentuales para cada una de las estimaciones. Como es de esperar, la precisión de las predicciones es aceptable, aunque hay diferenias importantes en aquellos desarrollos para 1993 y 1995.

In [35]:
abs((Info_Estimada_1-Info_Real_1)/Info_Real_1)*100

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1989,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018257
1990,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.477673,4.489351
1991,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.827310,4.269424,5.834606
1992,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,3.522644,6.783678,10.196513,12.777367
1993,0.0,0.00000,0.000000,0.000000,0.000000,4.126469,8.042790,11.204964,14.356125,16.895881
1994,0.0,0.00000,0.000000,0.000000,0.010265,1.114509,3.361387,4.865206,6.981588,8.600395
1995,0.0,0.00000,0.000000,2.306833,4.918728,7.681654,10.422485,12.906863,15.641882,17.840191
1996,0.0,0.00000,2.513417,4.090412,4.752501,4.875502,3.449271,1.894176,0.276049,1.948717


#Modelo de regresión

Con base en las clases creadas, se presentarán algunas métricas de evaluación para las regresiones del modelo multiplicativo (Bases modelo multiplicativo), así como las del ejercicio de regresiones Ridge y Lasso. En adición, se llevará a cabo un análisis de Leave One Out Cross Validation.

#Regresión lineal para Bases Modelo Multiplicativo
La evaluación del desempeño para el primer modelo de Bases Modelo Multiplicativo se hace por error cuadrático medio, este es un método muy directo y se lleva a cabo de la siguiente manera:

In [36]:
mse = mean_squared_error(y_prueba, y_pred)
print("El error cuadrático medio es ", round(mse*100,3), " %")

El error cuadrático medio es  4.447  %


#Regresión Lineal Ridge y Lasso

En esta sub sección, se ven los resultados de los modelos de regresión Ridge y Lasso derivados en las secciones anteriores. Se usarán métricas como la del Error Porcentual Ajustado Medio.

En esta parte obtenemos los coeficientes estimados para las regresiones en cuestión, eso se ve de la siguiente manera:

In [37]:
resultados_regresion["coeficientes"]

{'Coef_normal': array([ 0.        , -0.05366319, -0.12604845, -0.22685027, -0.28797468,
        -0.35457203, -0.39991772, -0.49217904, -0.49546519, -0.51181813,
         0.29862739,  0.39832096,  0.53248843,  0.60872173,  0.63867745,
         0.75294324,  0.81777038,  0.84446757,  0.69437918]),
 'Coef_ridge1': array([ 0.        , -0.05364488, -0.12603211, -0.22683501, -0.28796046,
        -0.35455841, -0.39990494, -0.49216591, -0.49545364, -0.51180904,
         0.29858475,  0.39827697,  0.53244259,  0.60867406,  0.63862804,
         0.75288984,  0.8177119 ,  0.84440078,  0.69430212]),
 'Coef_lasso': array([ 0.        , -0.        , -0.07173795, -0.17254086, -0.23366693,
        -0.30026734, -0.34561888, -0.4378919 , -0.4412045 , -0.45764135,
         0.24350776,  0.34320307,  0.47737344,  0.55360948,  0.5835672 ,
         0.69783375,  0.76265966,  0.7893516 ,  0.6385957 ])}

Acá tenemos una vista a la matriz de diseño

In [38]:
resultados_regresion["X"]

,u,alpha1,alpha2,alpha3,alpha4,alpha5,alpha6,alpha7,alpha8,alpha9,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
766,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
767,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
768,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Se realiza la evaluación correspondiente

In [39]:
resultados_validacion1 = resultados1.predict(MedMal_filtrado_unico)

Error porcentual ajustado medio

In [40]:
resultados_validacion1[0]['Metricas MAPE']

{'MAPE': 17.862565448352345,
 'MAPE_ridge_1': 17.862732438617133,
 'MAPE_lasso': 17.86437860509435}

In [41]:
print("sSe evidencia que el mejor modelo por la métrica MAPE es:", resultados_validacion1[1]["nombre mejor modelo"])

sSe evidencia que el mejor modelo por la métrica MAPE es: Regresión normal


Se crea nueva variable para almacenar los nuevos resultados y mostrar la comparación de los modelos

In [42]:
resultados_test = resultados1.predict_test(MedMal_filtrado_unico1)


Aca se evaluan los resultados con base en las cifras conocidas dado el supuesto inicial de conocimiento de la información planteado al inicio del ejercicio

In [43]:
resultados_test["Comparacion"]

,Y_test,Y_ajustado modelo final,GRCODE
0,10.293365,9.751004,683
1,10.238637,9.828707,683
2,10.172522,9.678619,683
3,10.339837,9.701208,683
4,10.164274,9.727906,683
5,10.126511,9.577817,683
6,10.464873,9.575257,683
7,10.273464,9.640084,683
8,10.297993,9.666781,683
9,10.174926,9.516693,683


Se presenta la evaluación de estos nuevos resultados para los modelos existentes.

In [44]:
resultados_test["Metricas MAPE"]

{'MAPE modelo final': 8.848337795010787}

#Diseño de experimento usando Leave One Out Cross Validation

Con base en las validaciones anteriores, se considera pertinente evaluar el éxito de aquellos modelos donde se revisó la métrica MAPE.

El ejercicio se realizará para 10 aseguradoras por motivos de procesamiento



In [45]:
datos_LOOCV = MedMal_filtrado[MedMal_filtrado["GRCODE"].isin(list(MedMal_filtrado["GRCODE"].unique()[0:10]))]
datos_LOOCV

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_F2
0,669,Scpie Indemnity Co,1988,1988,1,121905
1,669,Scpie Indemnity Co,1988,1989,2,112211
2,669,Scpie Indemnity Co,1988,1990,3,103226
3,669,Scpie Indemnity Co,1988,1991,4,99599
4,669,Scpie Indemnity Co,1988,1992,5,96006
...,...,...,...,...,...,...
2895,40568,Seguros Triples Inc,1997,2002,6,3030
2896,40568,Seguros Triples Inc,1997,2003,7,3180
2897,40568,Seguros Triples Inc,1997,2004,8,3274
2898,40568,Seguros Triples Inc,1997,2005,9,3262


A continuación, se muestra la implementación del algorítmo para LOOCV, donde se busca hacer una comparación entre los modelos usados en las secciones anteriores y el modelo determinístico

In [46]:
#tiempo_inicial = time.time()
lista_aseguradoras = datos_LOOCV["GRCODE"].unique()
lista_aseguradoras
mejore_modelos_test_full = {}

for i in range(len(lista_aseguradoras)): #recorre las aseguradoras de test
    print("aseguradora de test:", i)
    conj_test = lista_aseguradoras[i] #codigo de aseguradora de testeo
    datos_test = datos_LOOCV[datos_LOOCV["GRCODE"].isin([conj_test])] #datos de aseguradora de testeo
    conj_entre_valid = np.delete(lista_aseguradoras, i, axis=0) #Conjunto de validación y entrenamiento

    mejores_modelos = []

    for j in range(len(conj_entre_valid)): #recorre los datos de entrenamiento y validación

        conj_vali = conj_entre_valid[j] #datos de aseguradora de validación
        conj_entre = np.delete(conj_entre_valid, j, axis=0) #aseguradoras de entrenamiento
        datos_train = datos_LOOCV[datos_LOOCV["GRCODE"].isin(conj_entre)] #datos de aseguradoras de entrenamiento
        datos_validacion = datos_LOOCV[datos_LOOCV["GRCODE"].isin([conj_vali])] #datos de aseguradora de validación

        #Se crea la clase que calculará las regresiones
        model1 = Reserva_Regresion_lineal(tabla = datos_train, origin = "AccidentYear", development = "DevelopmentLag", columns = datos_train.columns[5],
                                        index = "GRCODE", alpha = 0.001, matriz_de_ceros = matriz_de_ceros)

        #entrenamiento con conjunto de entrenamiento
        model1_regresion = model1.Regresion_lineal()

        #Se aplica la clase a los datos de validación
        model1_prediccion = model1.predict(datos_validacion) #acá el conjunto de validación escoge el mejor modelo de los tres de la clase

        #se aplica la clase a los datos de test
        #Acá se tienen las métricas evaluadas en el conjunto de teste del mejor modelo que el conjunto de validación escogió
        modelo_test = model1.predict_test(datos_test)

        mejore_modelos_test_full["modelo_"+str(i)+"-"+str(j)] = modelo_test #se guardan los mejores modelos

modelo_final = {}
for i in mejore_modelos_test_full.keys(): #recorre los mejores modelos finales
    modelo_final[i] = mejore_modelos_test_full[i]['Metricas MAPE']['MAPE modelo final']

nombre_modelo_final = list(dict(sorted(modelo_final.items(), key=lambda item: item[1])).keys())[0] #se ordenan los modelo de mejor a peor
Mejor_modelo_reserva = mejore_modelos_test_full[nombre_modelo_final] #se halla mejor modelo


aseguradora de test: 0
aseguradora de test: 1
aseguradora de test: 2
aseguradora de test: 3
aseguradora de test: 4
aseguradora de test: 5
aseguradora de test: 6
aseguradora de test: 7
aseguradora de test: 8
aseguradora de test: 9


Determinación del mejor modelo

In [47]:
Mejor_modelo_reserva["nombre mejor modelo"]

'Regresión de Lasso'

Coeficientes del mejor modelo

In [48]:
Mejor_modelo_reserva["coeficientes"]

array([ 0.        , -0.        , -0.07488313, -0.16424896, -0.2327577 ,
       -0.31436918, -0.37158089, -0.49418081, -0.50544788, -0.55232072,
        0.21127437,  0.29961793,  0.26306526,  0.36098568,  0.42668986,
        0.50299664,  0.58388548,  0.63958113,  0.5572926 ])

Comparación del mejor modelo

In [49]:
Mejor_modelo_reserva["Comparacion"]

,Y_test,Y_ajustado modelo final,GRCODE
0,9.852194,9.944153,36676
1,9.798127,9.951559,36676
2,9.827794,9.869270,36676
3,9.740969,9.806497,36676
4,9.740969,9.862193,36676
5,9.698920,9.779904,36676
6,9.740969,9.657099,36676
7,9.758462,9.737988,36676
8,9.605755,9.793684,36676
9,9.698920,9.711395,36676


Mejor Modelo para modelación

In [50]:
Mejor_modelo_reserva["mejor modelo"]

Lasso(alpha=0.001)

Métrica MAPE del mejor modelo con el conjunto de test

In [51]:
Mejor_modelo_reserva["Metricas MAPE"]

{'MAPE modelo final': 1.5342914600687543}

Cálculos del MAPE para modelo determinístico

In [52]:
MAPE1 = []
for i in datos_LOOCV["GRCODE"].unique():
    diferencia = abs((triangulos_resultados[i]["Triangulo_full_acumulado"]-triangulos_resultados[i]["Triangulo_estimado"])
                     /triangulos_resultados[i]["Triangulo_full_acumulado"]) #se aplica formula mape
    MAPE = diferencia.sum().sum()/45*100 #se divide por 45 debido a que son 45 datos del triangulo inferior
    MAPE1.append(MAPE) #se halla mape para cada asegurador
MAPE1 = np.array(MAPE1) #se promedian los mape
Mape_chain_ladder = MAPE1.mean()
print("Métrica MAPE con el método Chain-Ladder:",Mape_chain_ladder)

Métrica MAPE con el método Chain-Ladder: 7.689302482799469


Comparación de modelos

In [53]:
print("Métrica MAPE con el método Chain-Ladder:",Mape_chain_ladder)
print("Métrica MAPE con el modelo final:",Mejor_modelo_reserva["Metricas MAPE"]['MAPE modelo final'])

Métrica MAPE con el método Chain-Ladder: 7.689302482799469
Métrica MAPE con el modelo final: 1.5342914600687543
